In [32]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json
from dataloader import Twitch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
data = Twitch("DE")